# Extraindo Dados
___

Esse arquivo tem o objetivo de extrair dados do dataframe BSI e criar uma tabela que contenha os seguintes dados:

- o identificador do discente; 
- o sexo do discente;
- o ano em que o discente ingressou na instituição;
- o ano e período da última matrícula realizada pelo discente;
- total de sementres cursados pelo discente;
- um calcúlo de semestres cursados dividido pelo o número de semestre ideal do curso;
- a carga horária cumprida pelo discente;
- a carga horária cumprida pelo discente dividida pela carga horaria ideal;
- tempo relativo que o discente evadiu ou concluiu o curso;
- status atual do discente;
- número total de faltas de cada discente na disciplina que ele cursou;
- nota de cada discente nas disciplinas que ele cursou;
- média final de cada discente nas disciplinas que ele cursou;
- quantidade de vezes que o discente se matriculou nas disciplinas;
- quantidade de vezes que o discente se matriculou em cada disciplina;
- quantidade de vezes que o discente foi aprovado em cada disciplina;
- quantidade de vezes que o discente foi reprovado em cada disciplina.

Importando o Pandas e o Csv.

In [2820]:
import pandas as pd
import csv

Leitura do arquivo em csv e carregamento desses dados em um DataFrame usando o ponto e vírgula como separador.

In [2821]:
df_dados = pd.read_csv('dataframe-bsi-2009-2022.csv', sep=';')

In [2822]:
pd.set_option('display.max_columns', 50)

Listando as colunas do dataframe.

In [2823]:
df_dados.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status'],
      dtype='object')

In [2824]:
df_dados['media_final']=df_dados['media_final'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['media_final'] = df_dados['media_final'].astype('float64')

In [2825]:
df_dados['nota']=df_dados['nota'].apply(lambda x: str(x).replace(',' , '.'))
df_dados['nota'] = df_dados['nota'].astype('float64')

In [2826]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54584 entries, 0 to 54583
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   discente        54584 non-null  object 
 1   unidade         50972 non-null  float64
 2   nota            48321 non-null  float64
 3   media_final     47687 non-null  float64
 4   frequencia      50471 non-null  float64
 5   descricao       54584 non-null  object 
 6   ano             54584 non-null  int64  
 7   id_componente   54584 non-null  int64  
 8   nome            54584 non-null  object 
 9   ch_total        54584 non-null  int64  
 10  sexo            54584 non-null  int64  
 11  ano_nascimento  54584 non-null  int64  
 12  ano_ingresso    54584 non-null  int64  
 13  status          54584 non-null  object 
dtypes: float64(4), int64(6), object(4)
memory usage: 5.8+ MB


# Tratamento de Dados

Listando os dados NaN.

In [2827]:
df_dados.isnull().sum()

discente             0
unidade           3612
nota              6263
media_final       6897
frequencia        4113
descricao            0
ano                  0
id_componente        0
nome                 0
ch_total             0
sexo                 0
ano_nascimento       0
ano_ingresso         0
status               0
dtype: int64

Preenchendo os dados NaN da coluna *unidade* com 1.

In [2828]:
df_dados.loc[:, 'unidade'] = df_dados['unidade'].fillna(1)

Apagando da coluna *descricao* a condição *INDEFERIDO*

In [2829]:
df_dados = df_dados[df_dados['descricao'] != 'INDEFERIDO']
df_dados = df_dados[df_dados['descricao'] != 'EXCLUIDA']

Verificar os valores faltosos na coluna *nota* com base na condição da coluna *descricao*.

In [2830]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'nota'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'nota'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'nota'] = -7

Verificar os valores faltosos na coluna *media_final* com base na condição da coluna *descricao*.

In [2831]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'media_final'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'media_final'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'media_final'] = -7

Verificar os valores faltosos na coluna *frequencia* com base na condição da coluna *descricao*.

In [2832]:
condicao_cancelado = df_dados['descricao'] == 'CANCELADO'
df_dados.loc[condicao_cancelado, 'frequencia'] = -2

condicao_trancado = df_dados['descricao'] == 'TRANCADO'
df_dados.loc[condicao_trancado, 'frequencia'] = -4

condicao_desistencia = df_dados['descricao'] == 'DESISTENCIA'
df_dados.loc[condicao_desistencia, 'frequencia'] = -7

Contando os valores dos dados da coluna *descricao*.

In [2833]:
df_dados.descricao.value_counts()

descricao
APROVADO                            28307
APROVADO POR NOTA                    8178
REPROVADO                            5790
REPROVADO POR MÉDIA E POR FALTAS     3804
CANCELADO                            2730
TRANCADO                             1913
DESISTENCIA                           457
REPROVADO POR FALTAS                  138
REPROVADO POR NOTA                     99
REPROVADO POR NOTA E FALTA             42
Name: count, dtype: int64

Criar uma função para mapear a classificação da coluna *descricao*.

In [2834]:
def mapear_classificacao(descricao):
    mapeamento = {
        'APROVADO': 2,
        'APROVADO POR NOTA': 1,
        'REPROVADO': -1,
        'REPROVADO POR FALTAS': -1,
        'REPROVADO POR NOTA': -1,
        'REPROVADO POR MÉDIA E POR FALTAS': -2,
        'REPROVADO POR NOTA E FALTA': -2,
        'CANCELADO': -4,
        'EXCLUIDA': -4,
        'TRANCADO': -4,
        'DESISTENCIA': -5,
    }
    return mapeamento.get(descricao, None)

Aplicar a função de mapeamento e criar a coluna *classificacao*.

In [2835]:
df_dados.loc[:, 'classificacao'] = df_dados['descricao'].map(mapear_classificacao)

In [2836]:
df_dados.descricao.value_counts()

descricao
APROVADO                            28307
APROVADO POR NOTA                    8178
REPROVADO                            5790
REPROVADO POR MÉDIA E POR FALTAS     3804
CANCELADO                            2730
TRANCADO                             1913
DESISTENCIA                           457
REPROVADO POR FALTAS                  138
REPROVADO POR NOTA                     99
REPROVADO POR NOTA E FALTA             42
Name: count, dtype: int64

Fazendo um recorte da nossa análise, vamos começar por disciplinas obrigatórias do Bacharelado em Sistemas da Informação (BSI):

In [2837]:
lista_obrigatórias = [
                'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                'INTRODUÇÃO À INFORMÁTICA',
                'FUNDAMENTOS DE MATEMÁTICA',
                'LÓGICA',
                'TEORIA GERAL DA ADMINISTRAÇÃO',
                'PROGRAMAÇÃO',
                'CÁLCULO DIFERENCIAL E INTEGRAL',
                'TEORIA GERAL DOS SISTEMAS',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                'ESTRUTURA DE DADOS',
                'ÁLGEBRA LINEAR',
                'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                'PROGRAMAÇÃO WEB',
                'ARQUITETURA DE COMPUTADORES',
                'PROBABILIDADE E ESTATÍSTICA',
                'BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE I',
                'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                'SISTEMAS OPERACIONAIS',
                'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                'ENGENHARIA DE SOFTWARE II',
                'REDES DE COMPUTADORES',
                'EMPREENDEDORISMO EM INFORMÁTICA',
                'GESTÃO DE PROJETO DE SOFTWARE',
                'PROGRAMAÇÃO VISUAL',
                'MATEMÁTICA FINANCEIRA',
                'SISTEMAS DE APOIO À DECISÃO',
                'ÉTICA',
                ]
condição_nome = f"nome in {lista_obrigatórias}"
df_dados_filtrado = df_dados.query(condição_nome)

Filtrar os dados onde a coluna **unidade** foi preenchida com 1.

In [2838]:
#df_dados_filtrado = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

## Extraindo dados

In [2839]:
df_dados_filtrado.columns

Index(['discente', 'unidade', 'nota', 'media_final', 'frequencia', 'descricao',
       'ano', 'id_componente', 'nome', 'ch_total', 'sexo', 'ano_nascimento',
       'ano_ingresso', 'status', 'classificacao'],
      dtype='object')

### Colunas com as **Disciplinas**

Calculando a quantidade de vezes que cada discente cursou cada *disciplina*.

In [2840]:
filtrado_unidade = df_dados_filtrado[df_dados_filtrado['unidade'] == 1]

In [2841]:
quantidade_disciplinas = filtrado_unidade.groupby(['discente', 'nome']).size().reset_index(name='quantidade')


Pivotando as *disciplinas*.

In [2842]:
tabela_final = quantidade_disciplinas.pivot(index='discente', columns='nome', values='quantidade').reset_index()

In [2843]:
tabela_final

nome,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,NaN,1.0,NaN,3.0,2.0,1.0,NaN,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,1.0,2.0,1.0,NaN,1.0,1.0,1.0,2.0,1.0
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,2.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,NaN,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,014f0dec46fe7a9c5836527662e1df10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN


### Coluna com o **Ano de Ingresso**

Adicionando o *ano_ingresso* para cada discente.

In [2844]:
ano_ingresso_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'ano_ingresso']]
tabela_final = tabela_final.merge(ano_ingresso_discente, on='discente', how='left')

### Coluna com o **Sexo** do Discente

Definir a opção global para permitir o downcasting silencioso

In [2845]:
pd.set_option('future.no_silent_downcasting', True)

Alterando o conteúdo da coluna *sexo*.

In [2846]:
df_dados_filtrado.loc[:, 'sexo'] = df_dados_filtrado['sexo'].replace({'M': 0, 'F': 1})
df_dados_filtrado = df_dados_filtrado.infer_objects(copy=False)

Adicionando a coluna *sexo*.

In [2847]:
sexo_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'sexo']]
tabela_final = tabela_final.merge(sexo_discente, on='discente', how='left')

### Coluna com o **Status** do Discente

Alterando o conteúdo da coluna *status*.

In [2848]:
df_dados_filtrado['status'] = df_dados_filtrado['status'].replace({
    'CANCELADO': -1, 
    'ATIVO': 0, 'ATIVO - FORMANDO': 0,
    'FORMADO': 1,'CONCLUÍDO': 1})

Adicionando o *status* para cada discente.


In [2849]:
status_discente = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'status']]
tabela_final = tabela_final.merge(status_discente, on='discente', how='left')
tabela_final 

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,sexo,status
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2018,0,-1
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,NaN,1.0,NaN,3.0,2.0,1.0,NaN,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN,1.0,2.0,1.0,NaN,1.0,1.0,1.0,2.0,1.0,2018,0,-1
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,2.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,2011,0,-1
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,NaN,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1
4,014f0dec46fe7a9c5836527662e1df10,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,2022,0,0
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,2020,0,0
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,2014,0,1


### Coluna com o **classificacao** da Descrição

Adicionando o *classificacao* para cada discente.

In [2850]:
classificacao_descricao = df_dados_filtrado.drop_duplicates(subset=['discente'])[['discente', 'classificacao']]
tabela_final = tabela_final.merge(classificacao_descricao, on='discente', how='left')

### Coluna com a **Carga Horária Cumprida**

Agrupando por discente e somando a *carga horária*.

In [2851]:
df_carga_horaria_cumprida = df_dados_filtrado.groupby('discente')['ch_total'].sum().reset_index()

Adicionando a *carga horária cumprida* de cada discente e renomeando a coluna.

In [2852]:
df_carga_horaria_cumprida.rename(columns={'ch_total': 'ch_cumprida'}, inplace=True)
tabela_final = tabela_final.merge(df_carga_horaria_cumprida, on='discente', how='left')

Dividindo a *carga horária cumprida* do discente pela carga horária das disciplinas obrigatórias.

In [2853]:
tabela_final['ch_cumprida_dividida'] = tabela_final['ch_cumprida'] / 1830

### Coluna com a **Carga Horária Cumprida** com **Reprovação**

Filtrar as disciplinas *reprovadas*.

In [2854]:
reprovadas_status = [
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
]

In [2855]:
df_reprovadas = df_dados_filtrado[df_dados_filtrado['descricao'].isin(reprovadas_status)]

Agrupar por discente e somar a *carga horária das disciplinas reprovadas*.

In [2856]:
ch_reprovacao = df_reprovadas.groupby('discente')['ch_total'].sum().reset_index()

Renomear a coluna resultante.

In [2857]:
ch_reprovacao.columns = ['discente', 'ch_reprovacao']

Adicionando a *carga horária de reprovação* de cada discente.

In [2858]:
tabela_final = tabela_final.merge(ch_reprovacao, on='discente', how='left')

Substituindo NaN por 0 nas colunas.

In [2859]:
tabela_final = tabela_final.fillna(0)

In [2860]:
tabela_final

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,sexo,status,classificacao,ch_cumprida,ch_cumprida_dividida,ch_reprovacao
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,0,-1,-5,330,0.180328,0.0
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2018,0,-1,-2,6480,3.540984,2880.0
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2011,0,-1,-1,2250,1.229508,810.0
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,-1,6930,3.786885,1620.0
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,-1,-4,720,0.393443,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022,0,0,-1,1620,0.885246,810.0
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,2,7050,3.852459,1440.0
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2020,0,0,2,4050,2.213115,0.0
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2014,0,1,-1,8010,4.377049,1890.0


### Coluna com **taxa de reprovação**

In [2861]:
tabela_final['taxa_reprovacao'] = tabela_final['ch_reprovacao'] / tabela_final['ch_cumprida'] 

### Coluna com o **Semestre**

Função para calcular o *semestre* com base no *ano de ingresso* e o *ano cursado.*

In [2862]:
"""def calcular_semestre(ano_ingresso, ano_cursado):
    try:
        # Convertendo os anos para strings e ajustando para o formato correto
        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)
        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)
        
        ano_ingresso_num = int(ano_ingresso_str[:4])
        semestre_ingresso = int(ano_ingresso_str[4])
        
        ano_cursado_num = int(ano_cursado_str[:4])
        semestre_cursado = int(ano_cursado_str[4])
        
        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2
        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)
        
        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro
    except (IndexError, ValueError):
        return pd.NA  # Retorna NA se houver um problema com o formato dos anos"""

"def calcular_semestre(ano_ingresso, ano_cursado):\n    try:\n        # Convertendo os anos para strings e ajustando para o formato correto\n        ano_ingresso_str = str(ano_ingresso) + '1' if len(str(ano_ingresso)) == 4 else str(ano_ingresso)\n        ano_cursado_str = str(ano_cursado) + '1' if len(str(ano_cursado)) == 4 else str(ano_cursado)\n        \n        ano_ingresso_num = int(ano_ingresso_str[:4])\n        semestre_ingresso = int(ano_ingresso_str[4])\n        \n        ano_cursado_num = int(ano_cursado_str[:4])\n        semestre_cursado = int(ano_cursado_str[4])\n        \n        anos_passados = (ano_cursado_num - ano_ingresso_num) * 2\n        semestres_passados = anos_passados + (semestre_cursado - semestre_ingresso)\n        \n        return semestres_passados + 1  # +1 porque o semestre de ingresso é contado como o primeiro\n    except (IndexError, ValueError):\n        return pd.NA  # Retorna NA se houver um problema com o formato dos anos"

Corrigir os valores das colunas *ano_ingresso* e *ano*.

In [2863]:
"""df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)
df_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)"""

"df_dados_filtrado['ano_ingresso'] = df_dados_filtrado['ano_ingresso'].astype(str)\ndf_dados_filtrado['ano'] = df_dados_filtrado['ano'].astype(str)"

Aplicar a função para criar a coluna *semestre*.

In [2864]:
"""df_dados_filtrado['semestre'] = df_dados_filtrado.apply(
    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1
)"""

"df_dados_filtrado['semestre'] = df_dados_filtrado.apply(\n    lambda x: calcular_semestre(x['ano_ingresso'], x['ano']), axis=1\n)"

In [2865]:
"""# Agrupar por discente e disciplina e coletar os semestres
semestres_por_disciplina = df_dados_filtrado.groupby(['discente', 'nome'])['semestre'].apply(lambda x: ', '.join(map(str, sorted(x.unique())))).reset_index()

# Renomear a coluna para algo mais descritivo
semestres_por_disciplina.rename(columns={'semestre': 'semestres_cursados'}, inplace=True)

# Exibir o resultado
print(semestres_por_disciplina)"""

"# Agrupar por discente e disciplina e coletar os semestres\nsemestres_por_disciplina = df_dados_filtrado.groupby(['discente', 'nome'])['semestre'].apply(lambda x: ', '.join(map(str, sorted(x.unique())))).reset_index()\n\n# Renomear a coluna para algo mais descritivo\nsemestres_por_disciplina.rename(columns={'semestre': 'semestres_cursados'}, inplace=True)\n\n# Exibir o resultado\nprint(semestres_por_disciplina)"

In [2866]:
"""# Supondo que tabela_final já existe e tem as colunas 'discente' e 'disciplina'
tabela_final = tabela_final.merge(semestres_por_disciplina, on=['discente'], how='left')
tabela_final"""

"# Supondo que tabela_final já existe e tem as colunas 'discente' e 'disciplina'\ntabela_final = tabela_final.merge(semestres_por_disciplina, on=['discente'], how='left')\ntabela_final"

In [2867]:
"""semestres = df_dados_filtrado[['discente', 'semestre']].drop_duplicates("discente")
tabela_final = tabela_final.merge(semestres, on='discente', how='left')
tabela_final"""


'semestres = df_dados_filtrado[[\'discente\', \'semestre\']].drop_duplicates("discente")\ntabela_final = tabela_final.merge(semestres, on=\'discente\', how=\'left\')\ntabela_final'

In [2868]:
"""pivot_table = df_dados_filtrado.pivot_table(
    index=['discente', 'semestre'], 
    values=['frequencia', 'media_final'],
    aggfunc='first'
)
pivot_table"""

"pivot_table = df_dados_filtrado.pivot_table(\n    index=['discente', 'semestre'], \n    values=['frequencia', 'media_final'],\n    aggfunc='first'\n)\npivot_table"

In [2869]:
"""pivot_table2 = df_dados_filtrado.pivot_table(
    index=['discente'], 
    values=['frequencia', 'media_final', 'semestre'],
    aggfunc='first'
)
pivot_table2"""

"pivot_table2 = df_dados_filtrado.pivot_table(\n    index=['discente'], \n    values=['frequencia', 'media_final', 'semestre'],\n    aggfunc='first'\n)\npivot_table2"

In [2870]:
df_dados_filtrado

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54576,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1
54577,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1
54581,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2
54582,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2


In [2871]:
tabela_final.to_csv('tabela222.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [2872]:
tabela_final

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,SISTEMAS OPERACIONAIS,TEORIA GERAL DA ADMINISTRAÇÃO,TEORIA GERAL DOS SISTEMAS,ÁLGEBRA LINEAR,ÉTICA,ano_ingresso,sexo,status,classificacao,ch_cumprida,ch_cumprida_dividida,ch_reprovacao,taxa_reprovacao
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2018,0,-1,-5,330,0.180328,0.0,0.000000
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2018,0,-1,-2,6480,3.540984,2880.0,0.444444
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,2011,0,-1,-1,2250,1.229508,810.0,0.360000
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,-1,6930,3.786885,1620.0,0.233766
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2020,0,-1,-4,720,0.393443,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2022,0,0,-1,1620,0.885246,810.0,0.500000
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2016,0,1,2,7050,3.852459,1440.0,0.204255
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,2020,0,0,2,4050,2.213115,0.0,0.000000
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2014,0,1,-1,8010,4.377049,1890.0,0.235955


### Coluna com o Total de **Semestre Cursado**

Identificar *semestres* únicos cursados por cada discente.

In [2873]:
semestres_unicos_por_discente = df_dados_filtrado.groupby('discente')['ano'].nunique().reset_index()
semestres_unicos_por_discente.rename(columns={'ano' : 'semestre_total'}, inplace=True)

Adicionando a quantidade de *semestres* cursados por cada discente.

In [2874]:
tabela_final = tabela_final.merge(semestres_unicos_por_discente, on='discente', how='left')

Dividindo a quantidade de *semestres cursados* pela quantidade de *semestres estimados* no curso.

In [2875]:
tabela_final['semestre_dividido'] = tabela_final['semestre_total'] / 8

### Coluna com o **Último Período** Matrículado

Encontrar o *último período* que cada aluno estudou no curso.

In [2876]:
ultimo_periodo = df_dados_filtrado.groupby('discente')['ano'].max().reset_index()
ultimo_periodo.rename(columns={'ano': 'ultimo_periodo'}, inplace=True)

Adicionando as informações do *último período* de cada discente.

In [2877]:
tabela_final = tabela_final.merge(ultimo_periodo, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente se **Matriculou**

Soma dos valores das colunas para contabilizar a *matrícula* por discente.

In [2878]:
matricula =      tabela_final['ALGORITMOS E LÓGICA DE PROGRAMAÇÃO'] + \
                 tabela_final['INTRODUÇÃO À INFORMÁTICA'] + \
                 tabela_final['FUNDAMENTOS DE MATEMÁTICA'] + \
                 tabela_final['LÓGICA'] + \
                 tabela_final['TEORIA GERAL DA ADMINISTRAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO'] + \
                 tabela_final['CÁLCULO DIFERENCIAL E INTEGRAL'] + \
                 tabela_final['TEORIA GERAL DOS SISTEMAS'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS I'] + \
                 tabela_final['ESTRUTURA DE DADOS'] + \
                 tabela_final['ÁLGEBRA LINEAR'] + \
                 tabela_final['ORGANIZAÇÃO, SISTEMAS E MÉTODOS'] + \
                 tabela_final['FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO'] + \
                 tabela_final['PROGRAMAÇÃO WEB'] + \
                 tabela_final['ARQUITETURA DE COMPUTADORES'] + \
                 tabela_final['PROBABILIDADE E ESTATÍSTICA'] + \
                 tabela_final['BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE I'] + \
                 tabela_final['PROGRAMAÇÃO ORIENTADA A OBJETOS II'] + \
                 tabela_final['SISTEMAS OPERACIONAIS'] + \
                 tabela_final['PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS'] + \
                 tabela_final['ENGENHARIA DE SOFTWARE II'] + \
                 tabela_final['REDES DE COMPUTADORES'] + \
                 tabela_final['EMPREENDEDORISMO EM INFORMÁTICA'] + \
                 tabela_final['GESTÃO DE PROJETO DE SOFTWARE'] + \
                 tabela_final['PROGRAMAÇÃO VISUAL'] + \
                 tabela_final['MATEMÁTICA FINANCEIRA'] + \
                 tabela_final['SISTEMAS DE APOIO À DECISÃO'] + \
                 tabela_final['ÉTICA']

Adicionando a coluna *matrícula* na tabela.

In [2879]:
tabela_final['matricula'] = matricula

### Coluna com o **Tempo Relativo** que o discente passou no curso

Converter a coluna *ch_cumprida_dividida* para tipo numérico.

In [2880]:
tabela_final['semestre_dividido'] = pd.to_numeric(tabela_final['semestre_dividido'], errors='coerce')

Converter a coluna *status* para tipo numérico.

In [2881]:
tabela_final['status'] = pd.to_numeric(tabela_final['status'], errors='coerce')

Multiplicar os valores da coluna *ch_cumprida_dividida* pela coluna *status*.

In [2882]:
tabela_final['tempo_relativo'] = tabela_final['semestre_dividido'] * tabela_final['status']

### Coluna com a quantidade de vezes que o discente foi **Aprovado**

Listar todas as disciplinas.

In [2883]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *aprovações*.

In [2884]:
aprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'].isin([
    'APROVADO',
    'APROVADO POR NOTA',
])]

Agrupar por discente e nome da disciplina para contar as *aprovações*.

In [2885]:
aprovacoes_count = aprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_APROVADO*.

In [2886]:
aprovacoes_count.columns = [f"{disciplina}_APROVADO" for disciplina in aprovacoes_count.columns]

Mesclar as *aprovações* de volta ao DataFrame original.

In [2887]:
df_final = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(aprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final, on='discente', how='left')

### Coluna com a quantidade de vezes que o discente foi **Reprovado**

Listar todas as disciplinas.

In [2888]:
disciplinas = df_dados_filtrado['nome'].unique()

Criar uma tabela de *reprovações*.

In [2889]:
reprovacoes = df_dados_filtrado[df_dados_filtrado['descricao'].isin([
    'REPROVADO',
    'REPROVADO POR NOTA',
    'REPROVADO POR FALTAS',
    'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR MÉDIA E POR FALTAS'
])]

Agrupar por discente e nome da disciplina para contar as *reprovações*.

In [2890]:
reprovacoes_count = reprovacoes.groupby(['discente', 'nome']).size().unstack(fill_value=0)

Renomear as colunas para incluir *_REPROVADO*.

In [2891]:
reprovacoes_count.columns = [f"{disciplina}_REPROVADO" for disciplina in reprovacoes_count.columns]

In [2892]:
df_dados_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38090 entries, 0 to 54583
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   discente        38090 non-null  object 
 1   unidade         38090 non-null  float64
 2   nota            38090 non-null  float64
 3   media_final     38090 non-null  float64
 4   frequencia      38090 non-null  float64
 5   descricao       38090 non-null  object 
 6   ano             38090 non-null  int64  
 7   id_componente   38090 non-null  int64  
 8   nome            38090 non-null  object 
 9   ch_total        38090 non-null  int64  
 10  sexo            38090 non-null  int64  
 11  ano_nascimento  38090 non-null  int64  
 12  ano_ingresso    38090 non-null  int64  
 13  status          38090 non-null  object 
 14  classificacao   38090 non-null  int64  
dtypes: float64(4), int64(7), object(4)
memory usage: 4.6+ MB


Mesclar as *reprovações* de volta ao DataFrame original.

In [2893]:
df_final_reprovacao = df_dados_filtrado.drop_duplicates('discente').set_index('discente').join(reprovacoes_count, on='discente').fillna(0).reset_index()
tabela_final = tabela_final.merge(df_final_reprovacao, on='discente', how='left')

In [2894]:
df_dados_filtrado['ano'] = pd.to_numeric(df_dados_filtrado['ano'], errors='coerce').fillna(0).astype(int)

In [2895]:
df_dados_filtrado = df_dados_filtrado.drop_duplicates()
df_dados_filtrado.to_csv('df_dados_filtrado4.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)
df_dados_filtrado

,discente,unidade,nota,media_final,frequencia,descricao,ano,id_componente,nome,ch_total,sexo,ano_nascimento,ano_ingresso,status,classificacao
0,bad4e0ec61e635f382e7e32c32d5ad11,1.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
1,bad4e0ec61e635f382e7e32c32d5ad11,2.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
2,bad4e0ec61e635f382e7e32c32d5ad11,3.0,0.0,0.0,44.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,1,1991,2009,-1,-1
3,bee926721a5b23a098fdfb850bf8d5b5,1.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1
4,bee926721a5b23a098fdfb850bf8d5b5,2.0,0.0,0.0,52.0,REPROVADO POR FALTAS,20091,2037000,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,90,0,1983,2009,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54576,22f4aed4a073c5e9515a8669e9c102f3,2.0,6.9,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1
54577,22f4aed4a073c5e9515a8669e9c102f3,3.0,5.0,5.2,0.0,APROVADO POR NOTA,20222,62764,PROGRAMAÇÃO VISUAL,60,0,1984,2019,0,1
54581,943463d2de8c6a60ec5f5d959ba7f1ac,1.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2
54582,943463d2de8c6a60ec5f5d959ba7f1ac,2.0,10.0,10.0,0.0,APROVADO,20222,2054400,MATEMÁTICA FINANCEIRA,60,0,1999,2018,1,2


In [2896]:
tabela_final = tabela_final.drop_duplicates()
tabela_final.to_csv('tabela_final4.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)
tabela_final

,discente,ALGORITMOS E LÓGICA DE PROGRAMAÇÃO,ARQUITETURA DE COMPUTADORES,BANCO DE DADOS,CÁLCULO DIFERENCIAL E INTEGRAL,EMPREENDEDORISMO EM INFORMÁTICA,ENGENHARIA DE SOFTWARE I,ENGENHARIA DE SOFTWARE II,ESTRUTURA DE DADOS,FUNDAMENTOS DE MATEMÁTICA,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO,GESTÃO DE PROJETO DE SOFTWARE,INTRODUÇÃO À INFORMÁTICA,LÓGICA,MATEMÁTICA FINANCEIRA,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS",PROBABILIDADE E ESTATÍSTICA,PROGRAMAÇÃO,PROGRAMAÇÃO ORIENTADA A OBJETOS I,PROGRAMAÇÃO ORIENTADA A OBJETOS II,PROGRAMAÇÃO VISUAL,PROGRAMAÇÃO WEB,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS,REDES DE COMPUTADORES,SISTEMAS DE APOIO À DECISÃO,...,EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO,ENGENHARIA DE SOFTWARE I_REPROVADO,ENGENHARIA DE SOFTWARE II_REPROVADO,ESTRUTURA DE DADOS_REPROVADO,FUNDAMENTOS DE MATEMÁTICA_REPROVADO,FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO,GESTÃO DE PROJETO DE SOFTWARE_REPROVADO,INTRODUÇÃO À INFORMÁTICA_REPROVADO,LÓGICA_REPROVADO,MATEMÁTICA FINANCEIRA_REPROVADO,"ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO",PROBABILIDADE E ESTATÍSTICA_REPROVADO,PROGRAMAÇÃO_REPROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO,PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO,PROGRAMAÇÃO VISUAL_REPROVADO,PROGRAMAÇÃO WEB_REPROVADO,PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO,REDES DE COMPUTADORES_REPROVADO,SISTEMAS DE APOIO À DECISÃO_REPROVADO,SISTEMAS OPERACIONAIS_REPROVADO,TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO,TEORIA GERAL DOS SISTEMAS_REPROVADO,ÁLGEBRA LINEAR_REPROVADO,ÉTICA_REPROVADO
0,001cea3c82e2010681f2cdeab21e5ecf,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,005c14d7c07bf7980b60c703f99c5ee7,1.0,2.0,1.0,1.0,0.0,1.0,0.0,3.0,2.0,1.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,...,0.0,3.0,0.0,9.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,6.0,0.0
2,0107fd69d8cd7e3d30dede96fb68bfe5,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,014789363f7940922e71e710ee9d22bc,2.0,3.0,1.0,0.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,3.0,6.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,014f0dec46fe7a9c5836527662e1df10,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,fe802d8d85de6f842749468401d1146c,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
676,fe87dfa176a74fc10a5cb701b9fb5dd4,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,fec9ed6026d55ecdf514c640312c3d08,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
678,ff56f2c5048dae0797fd3e851572b80c,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,1.0,1.0,4.0,4.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,3.0,6.0,0.0,0.0,6.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2897]:
tabela_final = tabela_final.merge(algoritmo, on='discente', how='left')
tabela_final

NameError: name 'algoritmo' is not defined

Mudando a ordem das colunas.

In [ ]:
'''colunas_ordenadas = ['discente', 'sexo', 'ano_ingresso', 'ultimo_periodo', 'ch_cumprida',
                    'ch_cumprida_dividida', 'ch_reprovacao',
                    'taxa_reprovacao', 'semestre', 'semestre_dividido', 'tempo_relativo', 'status', 'matricula',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FREQUENCIA',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'INTRODUÇÃO À INFORMÁTICA',
                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_REPROVADO',
                    'INTRODUÇÃO À INFORMÁTICA_FREQUENCIA',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_1',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_2',
                    'INTRODUÇÃO À INFORMÁTICA_NOTA_3',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_1',
                    'INTRODUÇÃO À INFORMÁTICA_MEDIA_FINAL_2',
                    'FUNDAMENTOS DE MATEMÁTICA',
                    'FUNDAMENTOS DE MATEMÁTICA_APROVADO',
                    'FUNDAMENTOS DE MATEMÁTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'LÓGICA',
                    'LÓGICA_APROVADO',
                    'LÓGICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'TEORIA GERAL DA ADMINISTRAÇÃO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_APROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_REPROVADO',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_FREQUENCIA',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_1',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_2',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_NOTA_3',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_1',
                    'TEORIA GERAL DA ADMINISTRAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO',
                    'PROGRAMAÇÃO_APROVADO',
                    'PROGRAMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'CÁLCULO DIFERENCIAL E INTEGRAL',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_APROVADO',
                    'CÁLCULO DIFERENCIAL E INTEGRAL_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'TEORIA GERAL DOS SISTEMAS',
                    'TEORIA GERAL DOS SISTEMAS_APROVADO',
                    'TEORIA GERAL DOS SISTEMAS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS I_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ESTRUTURA DE DADOS',
                    'ESTRUTURA DE DADOS_APROVADO',
                    'ESTRUTURA DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ÁLGEBRA LINEAR',
                    'ÁLGEBRA LINEAR_APROVADO',
                    'ÁLGEBRA LINEAR_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_APROVADO',
                    'ORGANIZAÇÃO, SISTEMAS E MÉTODOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_APROVADO',
                    'FUNDAMENTOS DE SISTEMAS DE INFORMAÇÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO WEB',
                    'PROGRAMAÇÃO WEB_APROVADO',
                    'PROGRAMAÇÃO WEB_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ARQUITETURA DE COMPUTADORES',
                    'ARQUITETURA DE COMPUTADORES_APROVADO',
                    'ARQUITETURA DE COMPUTADORES_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROBABILIDADE E ESTATÍSTICA',
                    'PROBABILIDADE E ESTATÍSTICA_APROVADO',
                    'PROBABILIDADE E ESTATÍSTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'BANCO DE DADOS',
                    'BANCO DE DADOS_APROVADO',
                    'BANCO DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ENGENHARIA DE SOFTWARE I',
                    'ENGENHARIA DE SOFTWARE I_APROVADO',
                    'ENGENHARIA DE SOFTWARE I_REPROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_APROVADO',
                    'PROGRAMAÇÃO ORIENTADA A OBJETOS II_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'SISTEMAS OPERACIONAIS',
                    'SISTEMAS OPERACIONAIS_APROVADO',
                    'SISTEMAS OPERACIONAIS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_APROVADO',
                    'PROJETO E ADMINISTRAÇÃO DE BANCO DE DADOS_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ENGENHARIA DE SOFTWARE II',
                    'ENGENHARIA DE SOFTWARE II_APROVADO',
                    'ENGENHARIA DE SOFTWARE II_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'REDES DE COMPUTADORES',
                    'REDES DE COMPUTADORES_APROVADO',
                    'REDES DE COMPUTADORES_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'EMPREENDEDORISMO EM INFORMÁTICA',
                    'EMPREENDEDORISMO EM INFORMÁTICA_APROVADO',
                    'EMPREENDEDORISMO EM INFORMÁTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'GESTÃO DE PROJETO DE SOFTWARE',
                    'GESTÃO DE PROJETO DE SOFTWARE_APROVADO',
                    'GESTÃO DE PROJETO DE SOFTWARE_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'PROGRAMAÇÃO VISUAL',
                    'PROGRAMAÇÃO VISUAL_APROVADO',
                    'PROGRAMAÇÃO VISUAL_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'MATEMÁTICA FINANCEIRA',
                    'MATEMÁTICA FINANCEIRA_APROVADO',
                    'MATEMÁTICA FINANCEIRA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'SISTEMAS DE APOIO À DECISÃO',
                    'SISTEMAS DE APOIO À DECISÃO_APROVADO',
                    'SISTEMAS DE APOIO À DECISÃO_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2',
                    'ÉTICA',
                    'ÉTICA_APROVADO',
                    'ÉTICA_REPROVADO',
                    'PROGRAMAÇÃO_FREQUENCIA',
                    'PROGRAMAÇÃO_NOTA_1',
                    'PROGRAMAÇÃO_NOTA_2',
                    'PROGRAMAÇÃO_NOTA_3',
                    'PROGRAMAÇÃO_MEDIA_FINAL_1',
                    'PROGRAMAÇÃO_MEDIA_FINAL_2']
'''

"colunas_ordenadas = ['discente', 'sexo', 'ano_ingresso', 'ultimo_periodo', 'ch_cumprida',\n                    'ch_cumprida_dividida', 'ch_reprovacao',\n                    'taxa_reprovacao', 'semestre', 'semestre_dividido', 'tempo_relativo', 'status', 'matricula',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_APROVADO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_REPROVADO',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_FREQUENCIA',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_1',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_2',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_NOTA_3',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_1',\n                    'ALGORITMOS E LÓGICA DE PROGRAMAÇÃO_MEDIA_FINAL_2',\n                    'INTRODUÇÃO À INFORMÁTICA',\n                    'INTRODUÇÃO À INFORMÁTICA_APROVADO',\n               

Reordenando as colunas.

In [ ]:
'''tabela_final = tabela_final[colunas_ordenadas]
tabela_final'''

'tabela_final = tabela_final[colunas_ordenadas]\ntabela_final'

Para salvar o DataFrame no formato CSV com a separação por ';' e garantir que os dados numéricos estejam no formato correto.

In [ ]:
df_dados_filtrado.to_csv('df_filtrado.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
tabela_final.to_csv('tabela_final.csv', index=False, sep=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
"""# Criar cópia do DataFrame para evitar o aviso SettingWithCopyWarning
df_dados_filtrado = df_dados_filtrado.copy()

# Inicializar a nova coluna com valores padrão
df_dados_filtrado['nota_1'] = pd.NA
df_dados_filtrado['nota_2'] = pd.NA
df_dados_filtrado['nota_3'] = pd.NA

# Preencher as novas colunas com os valores das notas correspondentes à unidade
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 1, 'nota_1'] = df_dados_filtrado['nota']
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 2, 'nota_2'] = df_dados_filtrado['nota']
df_dados_filtrado.loc[df_dados_filtrado['unidade'] == 3, 'nota_3'] = df_dados_filtrado['nota']"""